In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Import libraries

In [6]:
# Standard import
import os
import requests

# Third party import

## Torch
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim

## Data manipulation
import pandas as pd
import numpy as np

## Machine learning libraries


## Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

## Other
from tqdm import tqdm

In [3]:
def get_device():
    """Check that MPS is available, or cuda"""

    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    print(f"Device {device} selected")

    return device

device = get_device()

Device mps selected


## Explore Dataset


In [19]:
DATA_PATH = os.path.join(os.getcwd(),"data")
AML_SMALL_DATASET = os.path.join(DATA_PATH,"LI-Small_Transs.csv")
URL_DATA = (
    "https://drive.google.com/uc?export=download&id=1vRNPIhFbI9aeOzP4QTausZv3xX8GJbOO"
)
URL_DATA_2 = "https://drive.google.com/uc?export=download&id=10BY_NplIfbA9X1pLqS6JedOSPvAR23W0"

In [10]:
def download_file(url, filename):
    """Download an URL to a file"""
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, "wb") as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

In [11]:
if not os.path.exists(AML_SMALL_DATASET):
    if not os.path.exists(DATA_PATH):
        try:
            os.makedirs(DATA_PATH)
        except OSError as error:  
            print(error)   

    download_file(URL_DATA, AML_SMALL_DATASET)
else:
    print("Data is downloaded!")
   
# !wget URL_DATA -o AML_SMALL_DATASET

In [31]:
!wget --no-check-certificate -qO- "https://drive.google.com/uc?export=download&id=10BY_NplIfbA9X1pLqS6JedOSPvAR23W0" -o AML_SMALL_DATASET

<!DOCTYPE html><html><head><title>Google Drive - Virus scan warning</title><meta http-equiv="content-type" content="text/html; charset=utf-8"/><style nonce="jVfnqD6C5KyLlZMEhEBACg">.goog-link-button{position:relative;color:#15c;text-decoration:underline;cursor:pointer}.goog-link-button-disabled{color:#ccc;text-decoration:none;cursor:default}body{color:#222;font:normal 13px/1.4 arial,sans-serif;margin:0}.grecaptcha-badge{visibility:hidden}.uc-main{padding-top:50px;text-align:center}#uc-dl-icon{display:inline-block;margin-top:16px;padding-right:1em;vertical-align:top}#uc-text{display:inline-block;max-width:68ex;text-align:left}.uc-error-caption,.uc-warning-caption{color:#222;font-size:16px}#uc-download-link{text-decoration:none}.uc-name-size a{color:#15c;text-decoration:none}.uc-name-size a:visited{color:#61c;text-decoration:none}.uc-name-size a:active{color:#d14836;text-decoration:none}.uc-footer{color:#777;font-size:11px;padding-bottom:5ex;padding-top:5ex;text-align:center}.uc-footer a

In [17]:
URL_DATA

'https://drive.google.com/uc?export=download&id=1vRNPIhFbI9aeOzP4QTausZv3xX8GJbOO'

In [9]:
df = pd.read_csv(AML_SMALL_DATASET)

In [10]:
df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:08,11,8000ECA90,11,8000ECA90,3.195403e+06,US Dollar,3.195403e+06,US Dollar,Reinvestment,0
1,2022/09/01 00:21,3402,80021DAD0,3402,80021DAD0,1.858960e+03,US Dollar,1.858960e+03,US Dollar,Reinvestment,0
2,2022/09/01 00:00,11,8000ECA90,1120,8006AA910,5.925710e+05,US Dollar,5.925710e+05,US Dollar,Cheque,0
3,2022/09/01 00:16,3814,8006AD080,3814,8006AD080,1.232000e+01,US Dollar,1.232000e+01,US Dollar,Reinvestment,0
4,2022/09/01 00:00,20,8006AD530,20,8006AD530,2.941560e+03,US Dollar,2.941560e+03,US Dollar,Reinvestment,0
...,...,...,...,...,...,...,...,...,...,...,...
6924044,2022/09/10 23:39,71696,81B2518F1,71528,81C0482E1,3.346900e-02,Bitcoin,3.346900e-02,Bitcoin,Bitcoin,0
6924045,2022/09/10 23:48,271241,81B567481,173457,81C0DA751,1.313000e-03,Bitcoin,1.313000e-03,Bitcoin,Bitcoin,0
6924046,2022/09/10 23:50,271241,81B567481,173457,81C0DA751,1.305800e-02,Bitcoin,1.305800e-02,Bitcoin,Bitcoin,0
6924047,2022/09/10 23:57,170558,81A2206B1,275798,81C1D5CA1,4.145370e-01,Bitcoin,4.145370e-01,Bitcoin,Bitcoin,0
